# Input & Label

In [1]:
from rtmag.paper.load import load_input_label, MyModel

In [2]:
input_files, label_files = load_input_label("/mnt/f/isee_dataset/12673")
len(input_files), len(label_files)

(389, 389)

In [3]:
idx = 106
input_files[idx]

PosixPath('/mnt/f/isee_dataset/12673/input/input_12673_20170905_000000.npz')

# Model prediction & Label (NLFFF, Potential)

In [4]:
from time import time

In [5]:
# mm = MyModel("/home/mgjeon/space/workspace/base/pino_CC_Hnorm_Unit_2/best_model.pt")
mm = MyModel("/home/mgjeon/space/workspace/base_important/uno_pi_cc_hnorm_unit_aug/model_75.pt")
# mm2 = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_unit_aug/model_60.pt", clip=100000)
# mm3 = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_1_unit_aug_ccc_square/model_50.pt", clip=100000)

Model loaded from epoch 80


In [6]:
start_time = time()
b = mm.get_pred(input_files[idx])
print(f"{time() - start_time} seconds")

0.69140625 seconds


In [7]:
start_time = time()
B = mm.get_label(label_files[idx])
print(f"{time() - start_time} seconds")

1.7544548511505127 seconds


In [8]:
start_time = time()
Bp = mm.get_pot(label_files[idx])
print(f"{time() - start_time} seconds")

1.6958997249603271 seconds


In [9]:
start_time = time()
dx, dy, dz, dV = mm.get_dV(input_files[idx])
print(f"{time() - start_time} seconds")

0.00437164306640625 seconds


In [10]:
dV

1.991674e+23

# Quantitative Evaluation

In [11]:
import rtmag.paper.metric as m

In [17]:
start_time = time()
res = m.evaluate(b, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.81
E_n'      : 0.52
E_m'      : 0.17
eps       : 1.10
1.0786705017089844 seconds


In [18]:
start_time = time()
res = m.evaluate_energy(b, B, Bp, dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 1.86
ref_E_1e33: 1.69
pot_E_1e33: 1.26
dV_1e23   : 1.99
pred_E_unit: 235100166934.71
ref_E_unit: 213618787255.97
pot_E_unit: 159275709872.87
C_vec     : 0.96
C_cs      : 0.81
E_n_prime : 0.52
E_m_prime : 0.17
eps       : 1.10
rel_l2_err: 0.30
2.1394150257110596 seconds


In [19]:
start_time = time()
res = m.evaluate_energy(b[:, :, :128, :], B[:, :, :128, :], Bp[:, :, :128, :], dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 1.85
ref_E_1e33: 1.69
pot_E_1e33: 1.26
dV_1e23   : 1.99
pred_E_unit: 233215676870.97
ref_E_unit: 212784219428.51
pot_E_unit: 158608732546.37
C_vec     : 0.96
C_cs      : 0.84
E_n_prime : 0.56
E_m_prime : 0.19
eps       : 1.10
rel_l2_err: 0.30
1.189460039138794 seconds


In [20]:
start_time = time()
res = m.evaluate_energy(b[:, :, :64, :], B[:, :, :64, :], Bp[:, :, :64, :], dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 1.79
ref_E_1e33: 1.66
pot_E_1e33: 1.24
dV_1e23   : 1.99
pred_E_unit: 225700509361.66
ref_E_unit: 209093575462.76
pot_E_unit: 156480883818.08
C_vec     : 0.96
C_cs      : 0.81
E_n_prime : 0.61
E_m_prime : 0.26
eps       : 1.08
rel_l2_err: 0.29
0.5880846977233887 seconds


In [21]:
(dz/1e8) * 64

40.815574645996094

In [22]:
start_time = time()
res = m.evaluate(b[:, :, :64, :], B[:, :, :64, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.81
E_n'      : 0.61
E_m'      : 0.26
eps       : 1.08
0.2671339511871338 seconds


In [23]:
start_time = time()
res = m.evaluate(b[128:-128, 64:-64, :128, :], B[128:-128, 64:-64, :128, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.97
C_cs      : 0.94
E_n'      : 0.68
E_m'      : 0.50
eps       : 1.08
0.14412808418273926 seconds


In [24]:
m.metrics_j(b)

(0.7895285833926029, 107.23153960080086, 56.121059920603834)

In [25]:
m.metrics_j(B)

(0.6989067050349445, 101.28473408225598, 20.23915785396699)

In [26]:
m.metrics_j(Bp)

(0.8579550409294465, 4.676497951764051, 4.255680868829668)

In [28]:
start_time = time()
bf = mm.fine_tuning(dx=dx/1e8, dy=dy/1e8, dz=dz/1e8, 
                    w_reg=0.01, w_bc=10, w_ff=1, w_div=1,
                    lr=1e-5, n_epoch=10)
print(f"{time() - start_time} seconds")

0
1
2
3
4
5
6
7
8
9
14.532980918884277 seconds


In [29]:
m.metrics_j(bf)

(0.7857528696793513, 94.6204181077637, 47.43225828168303)

In [30]:
start_time = time()
res = m.evaluate(bf, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.72
E_n'      : 0.51
E_m'      : 0.09
eps       : 1.10
1.1563880443572998 seconds
